In [1]:
import numpy as np
from keras import models

In [12]:
get_model_directory = lambda series, cut: f"models/{series}_series/{cut}_cut/"
get_model_name = lambda number: f"m_{number}.h5"
series_number = 1707

def get_model(cut_num, number):
    model_name = get_model_directory(series_number, cut_num) + get_model_name(number)
    return models.load_model(model_name, compile = False)  

In [3]:
from gtda.homology import VietorisRipsPersistence

In [4]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(matrix):
    n = matrix.shape[1]
    cos = lambda a, b: dot(a, b) / (norm(a) * norm(b))
    
    new_matrix = np.zeros([n, n])

    for i in range(n):
        for j in range(i+1):
            new_matrix[i, j] = 1 - cos(matrix[:, i], matrix[:, j])
            new_matrix[j, i] = new_matrix[i, j]

    return new_matrix

def model_cos(model, layer_num):
    layers = model.layers
    weight_matrix = layers[layer_num].get_weights()[0]
    return cosine_similarity(weight_matrix)

In [5]:
VR = VietorisRipsPersistence(metric="precomputed")

In [6]:
def get_model_cos (cut, num):
    model = get_model(cut, num)
    return [model_cos(model, i) for i in range(len(model.layers) - 1)]

def analyse_model(cut, num):
    return VR.fit_transform(get_model_cos(cut, num))    

In [7]:
from gtda.plotting import plot_diagram

In [8]:
cuts = [1, 2, 4, 8, 16, 32, 64, 128, 254, 1024, 2048]

In [13]:
import os 
import pickle 

tries = 30
# s = 1707

for layer in range(3): 
    for i in range(tries):
        for cut in cuts: 
            d = analyse_model(cut, i)
            filename = f"diagram/d_dir_{series_number}/cut{cut}/num{i}"
            os.makedirs(os.path.dirname(filename), exist_ok=True) 
            with open(filename, "wb") as save_file: 
                pickle.dump(d, save_file)


C:\Users\kanad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\kanad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\kanad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\k

In [ ]:
tries = 30
def get_all_diagrams_from(cut):
    all = [analyse_model(cut, num) for num in range(tries)]
    return [[all[i][layer] for i in range(tries)] for layer in range(3)]